In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set() 
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
sns.set_style('darkgrid')

In [2]:
airlines = "data/airlines.csv" #Irrelevant
airports = "data/airports.csv" #Boston Logan International Airport Code: BOS
flights = "data/flights.csv"
weather = "data/USW00014739.2015.csv"

## Process Flight Dataset

In [3]:
flight_df = pd.read_csv(flights)

/Users/xhk/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
bfdf = flight_df[flight_df["ORIGIN_AIRPORT"]=="BOS"]
bfdf = bfdf[bfdf["CANCELLED"]==False]
bfdf = bfdf[["MONTH", "DAY", "DEPARTURE_DELAY"]]
bfdf.dropna()
bfdf["DATE"] = 100 * (bfdf["MONTH"] - 1) + bfdf["DAY"]
u = pd.unique(bfdf["DATE"])
print(bfdf)

         MONTH  DAY  DEPARTURE_DELAY  DATE
65           1    1             -4.0     1
71           1    1             -5.0     1
104          1    1             -4.0     1
108          1    1             -3.0     1
121          1    1              6.0     1
...        ...  ...              ...   ...
5818439     12   31             24.0  1131
5818465     12   31             -7.0  1131
5818546     12   31            212.0  1131
5818731     12   31             -7.0  1131
5819004     12   31             -3.0  1131

[105193 rows x 4 columns]


In [5]:
df = pd.DataFrame(columns=["DEPARTURE_DELAY"])
df["DEPARTURE_DELAY"] = bfdf.groupby(["DATE"])["DEPARTURE_DELAY"].mean().values
df["DATE"] = u
df["MONTH"] = (df["DATE"] / 100).astype(int) + 1
df["DAY"] = (df["DATE"] % 100).astype(int)
df = df.drop(columns=["DATE"])

## Process Weather Dataset

In [6]:
wthr_df = pd.read_csv(weather)
print(wthr_df.columns)
wthr_df[['YEAR','MONTH','DAY']] = wthr_df.DATE.str.split("-",expand=True)
wthr_df = wthr_df.drop(columns=['STATION','DATE','YEAR'])
wthr_df['MONTH'] = wthr_df['MONTH'].astype('int')
wthr_df['DAY'] = wthr_df['DAY'].astype('int')
wthr_df = wthr_df.replace(np.nan, 0)
wthr_df.head()

Index(['STATION', 'DATE', 'AWND', 'PRCP', 'SNOW', 'TAVG', 'TMAX', 'TMIN',
       'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09'],
      dtype='object')


,AWND,PRCP,SNOW,TAVG,TMAX,TMIN,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,MONTH,DAY
0,14.32,0.00,0.0,26,33,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,12.75,0.00,0.0,34,41,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2
2,10.29,0.62,1.4,28,37,22,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,3
3,8.28,0.57,0.0,38,52,35,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4
4,22.15,0.00,0.0,40,50,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5


## Merge Dataset on Time

In [7]:
mdf = pd.merge(wthr_df, df, on=['MONTH', 'DAY'])
mdf["DEPARTURE_DELAY"] = mdf["DEPARTURE_DELAY"].astype(int)
mdf.head()
mdf.to_csv("merged.csv", encoding='utf-8', index=False)